In [1]:
import os
import gc
import warnings
warnings.filterwarnings('ignore')
import random
import scipy as sp
import numpy as np
import pandas as pd
import joblib
import itertools
from tqdm.auto import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from itertools import combinations

import lightgbm as lgbm
from lightgbm import early_stopping
import seaborn as sns
import matplotlib.pyplot as plt
import optuna  # pip install optuna
from sklearn.metrics import log_loss

# **Train dataset**

In [2]:
# load train data set
# https://www.kaggle.com/datasets/raddar/amex-data-integer-dtypes-parquet-format
train = pd.read_parquet('C:\\Users\\16122\\AMEX Kaggle Competition\\train.parquet')

In [3]:
# define a list that includes all features except 'customer_ID' and 'S_2'
all_cols = [col for col in list(train.columns) if col not in ['customer_ID','S_2']]

# define a list of catagorical features that were provided by AMEX
cat_features = ["B_30","B_38","D_114","D_116","D_117","D_120","D_126","D_63","D_64","D_66","D_68"]

# define a list of numerical features
num_features = [col for col in all_cols if col not in cat_features]

In [4]:
# change feature 'S_2' to datetime
train['S_2'] = pd.to_datetime(train['S_2'])

In [5]:
# define a function that returns a list of numerical features with more than (threshold*100)% of null values
def find_null_num(df, threshold):
    nullvals = df[num_features].isnull().sum() / df.shape[0]
    etnullCols = nullvals[nullvals>threshold].index.to_list()
    return etnullCols

In [6]:
# 'etnullCols' contains numerical features with more than 90% null values 
etnullCols = find_null_num(train, 0.9)
etnullCols

['D_73', 'B_29', 'D_88', 'D_110', 'B_39', 'B_42', 'D_132', 'D_134']

In [7]:
# create new features that are differences between last and lag1 
# kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977

def get_difference(data, num_features):
    df1 = []
    customer_ids = []
    for customer_id, df in tqdm(data.groupby(['customer_ID'])):
        # Get the differences
        diff_df1 = df[num_features].diff(1).iloc[[-1]].values.astype(np.float32)
        # Append to lists
        df1.append(diff_df1)
        customer_ids.append(customer_id)
    # Concatenate
    df1 = np.concatenate(df1, axis = 0)
    # Transform to dataframe
    df1 = pd.DataFrame(df1, columns = [col + '_diff1' for col in df[num_features].columns])
    # Add customer id
    df1['customer_ID'] = customer_ids
    return df1

In [8]:
train_diff = get_difference(train, num_features)

100%|██████████| 458913/458913 [06:18<00:00, 1213.92it/s]


Now, we define different types of categorical features. We are going to handle them differently. 

In [9]:
# define features that have low cardinality
# https://www.kaggle.com/code/illidan7/amex-basic-feature-engineering-1500-features

# cat2 features (Low cardinality features; <=4 unique values)
cat2_features = ['B_31','B_32','B_33','D_103','D_109','D_111','D_127',
                'D_129','D_135','D_137','D_139','D_140','D_143','D_86',
                'D_87','D_92','D_93','D_94','D_96','R_15','R_19','R_2','R_21',
                'R_22','R_23','R_24','R_25','R_28','R_4','S_18','S_20','S_6']

# cat3 features (Low cardinality features; >=8 and <=21 unique values)
cat3_features = ['R_9','R_18','R_10','R_11','D_89','D_91','D_81','D_82','D_136',
                'D_138','D_51','D_123','D_125','D_108','B_41','B_22',]

In [10]:
# check that there is no overlap between etnullCols (columms with more than 90% null values)
# and cat2_features+cat3_features (low cardinality features)
over_lap = []
for col in etnullCols:
    if col in cat2_features or col in cat3_features:
        over_lap.append(col)
over_lap

[]

In [11]:
# define a list of numerical features that exclude low cardinality features
non_num = etnullCols + cat2_features + cat3_features
num_fea = [col for col in num_features if col not in non_num]

In [12]:
def process_and_feature_engineer(df):
    # FEATURE ENGINEERING FROM 
    # https://www.kaggle.com/code/huseyincot/amex-agg-data-how-it-created
    
    #create 'mean', 'std', 'min', 'max', 'last' columns for numerical features
    train_num_agg = df.groupby("customer_ID")[num_fea].agg(['mean', 'std', 'min', 'max', 'last'])
    train_num_agg.columns = ['_'.join(x) for x in train_num_agg.columns]
    train_num_agg.reset_index(inplace = True)
    
    #create 'count', 'last', 'nunique' for categorical features
    train_cat_agg = df.groupby("customer_ID")[cat_features].agg(['count', 'last', 'nunique'])
    train_cat_agg.columns = ['_'.join(x) for x in train_cat_agg.columns]
    train_cat_agg.reset_index(inplace = True)
    
    #create only 'last' for exnullCols
    train_etnull_agg = df.groupby("customer_ID")[etnullCols].agg(['last'])
    train_etnull_agg.columns = ['_'.join(x) for x in train_etnull_agg.columns]
    
    #create 'last' and 'unique' for cat2_features
    train_cat2_agg = df.groupby("customer_ID")[cat2_features].agg(['last', 'nunique'])
    train_cat2_agg.columns = ['_'.join(x) for x in train_cat2_agg.columns]
    
    #cat3_features are treated like num_fea but with one more 'nunique' column
    train_cat3_agg = df.groupby("customer_ID")[cat3_features].agg(['last', 'nunique','min', 'max','mean', 'std'])
    train_cat3_agg.columns = ['_'.join(x) for x in train_cat3_agg.columns]
    
    #merge all together
    df = train_num_agg.merge(train_cat_agg, how = 'inner', on = 'customer_ID').merge(train_etnull_agg, how = 'inner', on = 'customer_ID').merge(train_cat2_agg, how = 'inner', on = 'customer_ID').merge(train_cat3_agg, how = 'inner', on = 'customer_ID')
    del train_num_agg, train_cat_agg, train_etnull_agg, train_cat2_agg, train_cat3_agg
    print('shape after engineering', df.shape )
    
    return df

In [13]:
train = process_and_feature_engineer(train)

shape after engineering (458913, 807)


In [14]:
last_mean_col = cat3_features + num_fea

In [15]:
# create features that are differences between last and mean 
# kaggle.com/code/ragnar123/amex-lgbm-dart-cv-0-7977
def last_mean_diff(df,num_cols):
    for col in num_cols:
        try:
            df[f'{col}_last_mean_diff'] = df[f'{col}_last'] - df[f'{col}_mean']
        except:
            pass

In [16]:
last_mean_diff(train, last_mean_col)

In [17]:
# create features that are (max-last)/(max-min)
def last_max_ratio(df, num_cols):
    for col in num_cols:
        try:
            df[f'{col}_last_max_ratio'] = (df[f'{col}_max']-df[f'{col}_last'])/(df[f'{col}_max']-df[f'{col}_min'])
        except:
            pass

In [18]:
last_max_ratio(train, last_mean_col)

In [21]:
# there are some columns that contain inf values 
train.replace([np.inf, -np.inf], 0, inplace=True)

In [22]:
train = train.merge(train_diff, how = 'left', on = 'customer_ID')

In [23]:
train.head()

,customer_ID,P_2_mean,P_2_std,P_2_min,P_2_max,P_2_last,D_39_mean,D_39_std,D_39_min,D_39_max,...,D_136_diff1,D_137_diff1,D_138_diff1,D_139_diff1,D_140_diff1,D_141_diff1,D_142_diff1,D_143_diff1,D_144_diff1,D_145_diff1
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,0.933824,0.024194,0.868580,0.960384,0.934745,0.230769,0.832050,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-0.003376,0.0
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,0.899820,0.022119,0.861109,0.929122,0.880519,7.153846,6.743468,0,19,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000641,0.0
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,0.878454,0.028911,0.797670,0.904482,0.880875,0.000000,0.000000,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,-0.006491,0.0
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,0.598969,0.020107,0.567442,0.623392,0.621776,1.538462,3.017046,0,9,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000741,0.0
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,0.891679,0.042325,0.805045,0.940382,0.871900,0.000000,0.000000,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.000618,0.0


In [ ]:
# load target label file
targets = pd.read_csv('C:\\Users\\16122\\AMEX Kaggle Competition\\train_labels.csv')

In [ ]:
train = train.merge(targets, how = 'left', on = 'customer_ID')

In [ ]:
del targets
gc.collect()

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
%%time
new_cols = [col for col in list(train.columns) if col not in ['customer_ID','S_2','target']]
for col in new_cols:
    if train[col].dtype == 'float16' or train[col].dtype == 'float64':
        train[col] = train[col].astype('float32')

In [ ]:
for col in new_cols:
    if train[col].dtype == 'int64':
        train[col] = train[col].astype('int8')

In [ ]:
train.info()

In [ ]:
train.to_parquet('C:\\Users\\16122\\AMEX Kaggle Competition\\train_newnn_fe.parquet')

In [ ]:
del train
gc.collect()

# **Test dataset**

In [ ]:
test = pd.read_parquet('C:\\Users\\16122\\AMEX Kaggle Competition\\test.parquet')

In [ ]:
test['S_2']=pd.to_datetime(test['S_2'])

In [ ]:
test_diff = get_difference(test, num_features)

In [ ]:
test = process_and_feature_engineer(test)

In [ ]:
last_mean_diff(test, last_mean_col)

In [ ]:
last_max_ratio(test, last_mean_col)

In [ ]:
test.replace([np.inf, -np.inf], 0, inplace=True)

In [ ]:
test = test.merge(test_diff, how = 'left', on = 'customer_ID')

In [ ]:
del test_diff
gc.collect()

In [ ]:
test.head()

In [ ]:
%%time
new_cols = [col for col in list(test.columns) if col not in ['customer_ID','S_2']]
for col in new_cols:
    if test[col].dtype == 'float16' or test[col].dtype == 'float64':
        test[col] = test[col].astype('float32')

In [ ]:
for col in new_cols:
    if test[col].dtype == 'int16' or test[col].dtype == 'int64':
        test[col] = test[col].astype('int8')

In [ ]:
test.to_parquet('C:\\Users\\16122\\AMEX Kaggle Competition\\test_newnn_fe.parquet')

In [ ]:
del test
gc.collect()